In [2]:
import os

In [4]:
os.listdir("WIT-UAV-Dataset")

['2022-11-08_FIRE-SGL-108-Reade-Township_M600-Coverage-Test',
 '2021-11-08_FIRE-SGL-174-Rossiter_M100-Data-Collect',
 '2022-05-11_FIRE-SGL-111-Confluence_M600-Data-Collect']

- seek cameras
- FLIR cameras


per folder: 
- how many total boxes per class?
- what percentage of detections to no detections?
- visualization of detection number over time?
- size statistics of boxes?
- height of UAS?

how to ensure similar distributions of fire vs no fire?

```
WIT-UAV-Dataset
├── 2021-11-08_FIRE-SGL-174-Rossiter_M100-Data-Collect
│   ├── 2021-11-07_19-19-31_dji_sdk_and_thermal.repaired
│   ├── 2021-11-07_19-19-36_dji_sdk_and_thermal.repaired
│   ├── 2021-11-07_19-19-54_dji_sdk_and_thermal
│   └── 2021-11-07_19-20-26_dji_sdk_and_thermal.repaired
├── 2022-05-11_FIRE-SGL-111-Confluence_M600-Data-Collect
│   ├── seek_displayImage
│   │   ├── 2022-04-27_00-18-06
│   │   ├── 2022-04-27_00-18-47
│   │   └── 2022-04-27_00-24-53
│   └── thermal_1_image_rect
│       ├── 2022-04-27_00-18-06
│       ├── 2022-04-27_00-18-47
│       └── 2022-04-27_00-24-53
└── 2022-11-08_FIRE-SGL-108-Reade-Township_M600-Coverage-Test
    ├── 2022-11-08_13-15-55_dji_sdk_and_thermal
    ├── 2022-11-08_14-13-12_dji_sdk_and_thermal
    └── 2022-11-08_14-40-21_dji_sdk_and_thermal.kinda-repaired-images-only_labeled
```

In [7]:
list_of_runs = []
for dirpath, dirnames, filenames in os.walk("WIT-UAV-Dataset"):
    if not dirnames:
        list_of_runs.append(dirpath)
        print(dirpath, "has 0 subdirectories and", len(filenames), "files")

WIT-UAV-Dataset/2022-11-08_FIRE-SGL-108-Reade-Township_M600-Coverage-Test/2022-11-08_14-13-12_dji_sdk_and_thermal has 0 subdirectories and 193 files
WIT-UAV-Dataset/2022-11-08_FIRE-SGL-108-Reade-Township_M600-Coverage-Test/2022-11-08_14-40-21_dji_sdk_and_thermal.kinda-repaired-images-only_labeled has 0 subdirectories and 660 files
WIT-UAV-Dataset/2022-11-08_FIRE-SGL-108-Reade-Township_M600-Coverage-Test/2022-11-08_13-15-55_dji_sdk_and_thermal has 0 subdirectories and 250 files
WIT-UAV-Dataset/2021-11-08_FIRE-SGL-174-Rossiter_M100-Data-Collect/2021-11-07_19-20-26_dji_sdk_and_thermal.repaired has 0 subdirectories and 207 files
WIT-UAV-Dataset/2021-11-08_FIRE-SGL-174-Rossiter_M100-Data-Collect/2021-11-07_19-19-36_dji_sdk_and_thermal.repaired has 0 subdirectories and 515 files
WIT-UAV-Dataset/2021-11-08_FIRE-SGL-174-Rossiter_M100-Data-Collect/2021-11-07_19-19-31_dji_sdk_and_thermal.repaired has 0 subdirectories and 417 files
WIT-UAV-Dataset/2021-11-08_FIRE-SGL-174-Rossiter_M100-Data-Collec

In [8]:
list_of_runs

['WIT-UAV-Dataset/2022-11-08_FIRE-SGL-108-Reade-Township_M600-Coverage-Test/2022-11-08_14-13-12_dji_sdk_and_thermal',
 'WIT-UAV-Dataset/2022-11-08_FIRE-SGL-108-Reade-Township_M600-Coverage-Test/2022-11-08_14-40-21_dji_sdk_and_thermal.kinda-repaired-images-only_labeled',
 'WIT-UAV-Dataset/2022-11-08_FIRE-SGL-108-Reade-Township_M600-Coverage-Test/2022-11-08_13-15-55_dji_sdk_and_thermal',
 'WIT-UAV-Dataset/2021-11-08_FIRE-SGL-174-Rossiter_M100-Data-Collect/2021-11-07_19-20-26_dji_sdk_and_thermal.repaired',
 'WIT-UAV-Dataset/2021-11-08_FIRE-SGL-174-Rossiter_M100-Data-Collect/2021-11-07_19-19-36_dji_sdk_and_thermal.repaired',
 'WIT-UAV-Dataset/2021-11-08_FIRE-SGL-174-Rossiter_M100-Data-Collect/2021-11-07_19-19-31_dji_sdk_and_thermal.repaired',
 'WIT-UAV-Dataset/2021-11-08_FIRE-SGL-174-Rossiter_M100-Data-Collect/2021-11-07_19-19-54_dji_sdk_and_thermal',
 'WIT-UAV-Dataset/2022-05-11_FIRE-SGL-111-Confluence_M600-Data-Collect/seek_displayImage/2022-04-27_00-24-53',
 'WIT-UAV-Dataset/2022-05-11_

In [40]:
run = list_of_runs[1]

In [41]:
run

'WIT-UAV-Dataset/2022-11-08_FIRE-SGL-108-Reade-Township_M600-Coverage-Test/2022-11-08_14-40-21_dji_sdk_and_thermal.kinda-repaired-images-only_labeled'

In [47]:
def count_detections_per_class(path):
    num_v = 0
    num_h = 0
    with open(path, "r") as f:
        lines = f.readlines()
        for line in lines:
            if "v" in line:
                num_v += 1
            elif "h" in line:
                num_h += 1
            else:
                print(lines)
                raise ValueError(f"File {path} has no class")
    return num_v, num_h

In [48]:
def count_run_stats(run):
    num_total = 0
    num_labeled = 0
    total_num_v = 0
    total_num_h = 0
    for f in os.listdir(run):
        if "label" in f: # is label file
            num_v, num_h = count_detections_per_class(run + "/" + f)
            total_num_v += num_v
            total_num_h += num_h

            if num_v > 0 or num_h > 0:
                num_labeled += 1
        else: # is image file
            num_total += 1

    print(f"{num_total=}, {num_labeled=}, {total_num_v=}, {total_num_h=}")
    print(f"percentage labeled={num_labeled/num_total}")

In [50]:
for run in list_of_runs:
    print(run)
    count_run_stats(run)

WIT-UAV-Dataset/2022-11-08_FIRE-SGL-108-Reade-Township_M600-Coverage-Test/2022-11-08_14-13-12_dji_sdk_and_thermal
num_total=112, num_labeled=28, total_num_v=64, total_num_h=160
percentage labeled=0.25
WIT-UAV-Dataset/2022-11-08_FIRE-SGL-108-Reade-Township_M600-Coverage-Test/2022-11-08_14-40-21_dji_sdk_and_thermal.kinda-repaired-images-only_labeled
num_total=330, num_labeled=53, total_num_v=73, total_num_h=11
percentage labeled=0.1606060606060606
WIT-UAV-Dataset/2022-11-08_FIRE-SGL-108-Reade-Township_M600-Coverage-Test/2022-11-08_13-15-55_dji_sdk_and_thermal
num_total=130, num_labeled=24, total_num_v=3, total_num_h=76
percentage labeled=0.18461538461538463
WIT-UAV-Dataset/2021-11-08_FIRE-SGL-174-Rossiter_M100-Data-Collect/2021-11-07_19-20-26_dji_sdk_and_thermal.repaired
num_total=150, num_labeled=54, total_num_v=0, total_num_h=137
percentage labeled=0.36
WIT-UAV-Dataset/2021-11-08_FIRE-SGL-174-Rossiter_M100-Data-Collect/2021-11-07_19-19-36_dji_sdk_and_thermal.repaired
num_total=469, num

ValueError: File WIT-UAV-Dataset/2022-05-11_FIRE-SGL-111-Confluence_M600-Data-Collect/seek_displayImage/2022-04-27_00-18-47/time_0386.09.label has no class